In [434]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.font_manager as fm

# 讀取 CSV 文件
df = pd.read_csv('/Users/ccit0915/Desktop/kaggle/房價回歸預測/train_df.csv', encoding='utf-8')
df

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [436]:
# 計算 MasVnrType 的眾數（最常見的類別）
mas_vnr_mode = df['MasVnrType'].mode()[0]

# 計算 MasVnrArea 的平均值（去除 NaN）
mas_vnr_mean = df['MasVnrArea'].mean()

# 條件 1: 如果 MasVnrType 是 NaN，但 MasVnrArea 有值，則補上眾數
df.loc[df['MasVnrType'].isna() & df['MasVnrArea'].notna(), 'MasVnrType'] = mas_vnr_mode

# 條件 2: 如果 MasVnrType 是 NaN，且 MasVnrArea 也是 NaN，則 MasVnrType 補 'None'
df.loc[df['MasVnrType'].isna() & df['MasVnrArea'].isna(), 'MasVnrType'] = 'None'

# 條件 3: 如果 MasVnrArea 是 NaN，但 MasVnrType 有類別，則補上該欄的均值
df.loc[df['MasVnrArea'].isna() & df['MasVnrType'].notna(), 'MasVnrArea'] = mas_vnr_mean

missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0]
print(missing_values)

Series([], dtype: int64)


In [438]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

# 確保欄位名稱無多餘空格
df.columns = df.columns.str.strip()

# 分離特徵變數與目標變數
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

# 分割訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **步驟 1: 動態獲取 ordinal_cols 的類別**
ordinal_cols = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
    'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'FireplaceQu',
    'GarageFinish', 'GarageQual', 'GarageCond'
]

# 根據數據動態建立每個變數的類別順序
ordinal_categories = {col: sorted(X_train[col].dropna().unique().tolist()) for col in ordinal_cols}

# **步驟 2: Ordinal Encoding**
ordinal_encoder = OrdinalEncoder(categories=list(ordinal_categories.values()),
                                 handle_unknown='use_encoded_value',
                                 unknown_value=-1)

# 執行編碼
X_train[ordinal_cols] = ordinal_encoder.fit_transform(X_train[ordinal_cols])
X_test[ordinal_cols] = ordinal_encoder.transform(X_test[ordinal_cols])

# **步驟 3: 剩餘類別變數進行頻率編碼**
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in ordinal_cols]  # 移除已編碼的欄位

# 計算頻率並映射
for col in categorical_cols:
    freq_map = X_train[col].value_counts(normalize=True)  # 計算訓練集中每個類別的頻率
    X_train[col] = X_train[col].map(freq_map)
    X_test[col] = X_test[col].map(freq_map)  # 測試集應用相同映射

# 測試集中可能有 unseen 類別，填補 NaN 為 0
X_test[categorical_cols] = X_test[categorical_cols].fillna(0)

# **檢查是否還有非數值欄位**
non_numeric_X_train = X_train.select_dtypes(exclude=['number']).columns
non_numeric_X_test = X_test.select_dtypes(exclude=['number']).columns

# **確認目標變數是否數值型**
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)
non_numeric_y_train = y_train_df.select_dtypes(exclude=['number']).columns
non_numeric_y_test = y_test_df.select_dtypes(exclude=['number']).columns

# **輸出結果**
print("X_train 中的非數值欄位:", non_numeric_X_train.tolist())
print("X_test 中的非數值欄位:", non_numeric_X_test.tolist())
print("y_train 中的非數值欄位:", non_numeric_y_train.tolist())
print("y_test 中的非數值欄位:", non_numeric_y_test.tolist())

X_train 中的非數值欄位: []
X_test 中的非數值欄位: []
y_train 中的非數值欄位: []
y_test 中的非數值欄位: []


In [440]:
X_train_new = X_train.copy()

# 檢查缺失值
missing_values = X_train_new.isnull().sum()
print("缺失值檢查:")
print(missing_values[missing_values > 0])

# 檢查類別型資料
categorical_columns = X_train_new.select_dtypes(include=['object']).columns
print("\n類別型資料欄位:")
print(categorical_columns)

# 顯示類別型資料中每個欄位的唯一值數量
print("\n類別型資料欄位的唯一值數量:")
for col in categorical_columns:
    print(f"{col}: {X_train_new[col].nunique()} 唯一值")

缺失值檢查:
Series([], dtype: int64)

類別型資料欄位:
Index([], dtype='object')

類別型資料欄位的唯一值數量:


In [442]:
X_train = X_train.drop(columns=['Unnamed: 0', 'Id'])
X_test = X_test.drop(columns=['Unnamed: 0', 'Id'])

In [354]:
# 檢查 X_train 中的 Neighborhood 和 BsmtExposure 這兩個欄位的唯一值
neighborhood_values = X_train['Neighborhood'].unique()
bsmt_exposure_values = X_train['BsmtExposure'].unique()

# 顯示這兩個欄位的唯一值
print("Neighborhood 唯一值:", neighborhood_values)
print("BsmtExposure 唯一值:", bsmt_exposure_values)

Neighborhood 唯一值: [0.15496575 0.05565068 0.0744863  0.01797945 0.0984589  0.02226027
 0.05650685 0.05907534 0.04965753 0.03767123 0.07791096 0.0239726
 0.0385274  0.05222603 0.03424658 0.02825342 0.01284247 0.01113014
 0.00599315 0.00770548 0.01626712 0.01712329 0.00856164 0.00085616]
BsmtExposure 唯一值: [3. 0. 1. 2.]


In [394]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd

# 建立隨機森林模型
rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

# 訓練隨機森林模型
rf.fit(X_train, y_train)

# 獲取所有樣本在隨機森林中的葉節點索引
leaf_nodes = rf.apply(X_train)  # shape: (n_samples, n_trees)

# 轉換葉節點索引為字符串，形成條件子群
num_trees_to_use = 100  # 只使用前 3 棵樹的葉節點索引
leaf_nodes_str = np.apply_along_axis(lambda row: '_'.join(map(str, row[:num_trees_to_use])), axis=1, arr=leaf_nodes)

# 建立 DataFrame，並統計子群大小
leaf_df = pd.DataFrame({'leaf_group': leaf_nodes_str})
subgroup_counts = leaf_df['leaf_group'].value_counts()

# 查看子群大小分布
print(subgroup_counts)

leaf_group
6_21_24_12_15_8_12_6_9_9_6_9_9_23_9_23_6_8_21_15_9_13_6_9_6_8_6_23_23_6_6_23_6_23_12_9_6_21_6_13_6_6_12_9_9_9_21_21_9_9_6_13_9_9_9_8_9_6_6_6_23_15_15_9_9_6_13_9_15_9_9_13_6_9_9_24_15_9_6_9_13_9_23_8_6_9_8_21_9_9_6_6_9_23_9_15_6_9_8_8                            4
21_23_24_12_13_20_20_23_9_36_20_9_36_23_9_23_20_21_21_15_9_20_20_20_20_20_20_24_23_20_21_23_21_23_12_9_20_21_21_36_23_21_13_9_9_9_21_36_9_9_21_13_9_9_9_9_9_20_20_20_36_20_15_9_9_21_36_9_23_9_9_37_23_36_9_24_15_9_20_9_16_9_23_23_20_20_21_21_9_9_28_27_9_23_34    3
24_24_24_28_28_24_20_24_21_36_21_27_36_24_27_23_24_21_31_21_20_24_21_21_21_23_21_27_28_21_23_24_21_23_27_28_20_30_24_36_23_21_23_28_31_27_28_36_23_30_20_27_28_30_27_28_21_20_20_21_36_23_27_30_28_24_36_28_24_28_20_37_23_37_27_27_30_27_21_27_28_28_24_24_21_21    3
9_24_24_15_16_9_13_9_16_16_9_16_16_24_16_23_9_9_23_16_16_16_9_9_9_9_9_24_24_9_9_24_9_23_15_16_8_23_9_16_9_9_15_16_15_16_24_24_16_12_9_15_16_16_16_16_15_9_9_9_24_15_16_16_13_9_16_16_15_16_12_16_9_16_16

In [396]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# 計算 cPFI (conditional permutation feature importance)
def calculate_cPFI(rf, X, y, leaf_nodes, feature_idx, num_trees_to_use=100, min_group_size=2):
    """
    計算 cPFI (conditional permutation feature importance) 使用隨機森林
    :param rf: 已訓練的隨機森林模型
    :param X: 訓練資料
    :param y: 目標值
    :param leaf_nodes: 樹葉節點索引（數字格式）
    :param feature_idx: 要置換的特徵索引
    :param num_trees_to_use: 使用的樹數量
    :param min_group_size: 最小子群樣本數，避免樣本過少影響結果
    :return: 每個葉節點子群的 cPFI 值
    """
    # 取出每棵樹的葉節點索引，對每個樣本取第一棵樹的葉節點索引
    leaf_nodes_first_tree = leaf_nodes[:, 0]  # 或選擇 majority voting 的結果

    # 計算每個葉節點的樣本數
    subgroup_counts = pd.Series(leaf_nodes_first_tree).value_counts()

    # 過濾掉樣本數過少的葉節點
    valid_leaf_groups = subgroup_counts[subgroup_counts >= min_group_size].index

    # 初始化 cPFI 結果
    cPFI_values = {}

    # 根據葉節點子群計算 cPFI
    for leaf_group in valid_leaf_groups:
        # 找到屬於該葉節點的樣本索引
        group_mask = (leaf_nodes_first_tree == leaf_group)
        X_group = X[group_mask]
        y_group = y[group_mask]

        if X_group.shape[0] == 0:  # 確保該子群有數據
            continue

        # 計算原始模型的性能 (MSE)
        y_pred = rf.predict(X_group)
        original_mse = mean_squared_error(y_group, y_pred)

        # 生成特徵打亂後的 DataFrame
        X_group_shuffled = X_group.copy()
        X_group_shuffled.iloc[:, feature_idx] = np.random.permutation(X_group.iloc[:, feature_idx].values)  # 確保特徵被打亂

        # 計算置換後的 MSE
        y_pred_shuffled = rf.predict(X_group_shuffled)
        shuffled_mse = mean_squared_error(y_group, y_pred_shuffled)

        # 計算 cPFI (MSE 差異)
        cPFI_values[leaf_group] = shuffled_mse - original_mse

    return cPFI_values

# 計算所有特徵的 cPFI 並選出前 8 個重要特徵
def calculate_all_features_cPFI(rf, X, y, leaf_nodes, num_trees_to_use=100):
    """
    計算所有特徵的 cPFI，並選出前 8 個最重要的特徵
    :param rf: 已訓練的隨機森林模型
    :param X: 訓練資料
    :param y: 目標值
    :param leaf_nodes: 樹葉節點索引（數字格式）
    :return: 前 8 個最重要的特徵索引和其 cPFI 值
    """
    feature_cPFI = {}

    # 計算每個特徵的 cPFI
    for feature_idx in range(X.shape[1]):  # 遍歷所有特徵
        cPFI_values = calculate_cPFI(rf, X, y, leaf_nodes, feature_idx, num_trees_to_use=num_trees_to_use)
        if cPFI_values:
            avg_cPFI = np.mean(list(cPFI_values.values()))  # 計算該特徵在所有葉節點的平均 cPFI
        else:
            avg_cPFI = 0.0  # 如果該特徵沒有可用的 cPFI，則設為 0

        feature_cPFI[feature_idx] = avg_cPFI

    # 根據 cPFI 值排序，選出前 8 個最重要的特徵
    sorted_features = sorted(feature_cPFI.items(), key=lambda x: x[1], reverse=True)
    top_8_features = sorted_features[:8]

    return top_8_features

# 訓練隨機森林模型
rf = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_leaf=5, random_state=42)
rf.fit(X_train, y_train)

# 獲取葉節點索引
leaf_nodes = rf.apply(X_train)  # shape: (n_samples, n_trees)

# 計算 cPFI
top_8_features = calculate_all_features_cPFI(rf, X_train, y_train, leaf_nodes)

# 顯示最重要的前 8 個特徵及其 cPFI 值
print("Top 8 most important features based on cPFI:")
for idx, cPFI in top_8_features:
    print(f"Feature {idx}: cPFI = {cPFI}")

Top 8 most important features based on cPFI:
Feature 44: cPFI = 103547445.93339936
Feature 32: cPFI = 76025823.95802064
Feature 15: cPFI = 73453641.24174152
Feature 10: cPFI = 55028329.182703435
Feature 36: cPFI = 35996851.26343098
Feature 60: cPFI = 33614915.72849954
Feature 49: cPFI = 31796612.30125216
Feature 3: cPFI = 31743068.3848064


In [398]:
#num_trees_to_use = 100
feature_indices = [44, 32, 15, 10, 36, 60, 49, 3]
feature_names = X_train.columns[feature_indices]
print(feature_names)

Index(['GrLivArea', 'BsmtFinSF1', 'OverallQual', 'Neighborhood', 'TotalBsmtSF',
       'GarageArea', 'BedroomAbvGr', 'LotArea'],
      dtype='object')


In [406]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# 指定要使用的特徵
selected_features = ['GrLivArea', 'BsmtFinSF1', 'OverallQual', 'Neighborhood', 
                     'TotalBsmtSF', 'GarageArea', 'BedroomAbvGr', 'LotArea']

# 確保 'Neighborhood' 是數值型（如果未轉換，需進行編碼）
if X_train['Neighborhood'].dtype == 'object':
    freq_map = X_train['Neighborhood'].value_counts(normalize=True)
    X_train['Neighborhood'] = X_train['Neighborhood'].map(freq_map)
    X_test['Neighborhood'] = X_test['Neighborhood'].map(freq_map).fillna(0)

# 選取指定的特徵
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# 訓練隨機森林模型
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf.fit(X_train_selected, y_train)

# 進行預測
y_pred = rf.predict(X_test_selected)

# 計算評估指標
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 輸出結果
print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'MAE: {mae:.4f}')
print(f'R² Score: {r2:.4f}')

MSE: 792968787.9117
RMSE: 28159.7015
MAE: 18580.7955
R² Score: 0.8966


In [448]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# FK-RFE 函數：使用隨機森林評估特徵重要性
def fk_rfe(X_train, y_train, k):
    selected_features = list(X_train.columns)
    rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
    
    while len(selected_features) > k:
        rf.fit(X_train[selected_features], y_train)  # 訓練隨機森林回歸模型
        importances = rf.feature_importances_  # 獲取特徵重要性
        
        # 找到最不重要的特徵
        min_importance_index = np.argmin(importances)
        del selected_features[min_importance_index]  # 移除最不重要的特徵
    
    return selected_features


# 設定想選的特徵數量，例如 8
num_features = 8

# 執行 FK-RFE 選擇特徵
selected_features_fk_rfe = fk_rfe(X_train, y_train, num_features)
print("FK-RFE 選出的特徵:", selected_features_fk_rfe)

# 重新訓練 RF 回歸模型
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train[selected_features_fk_rfe], y_train)

# 預測
y_pred = rf.predict(X_test[selected_features_fk_rfe])

# 計算回歸評估指標
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 輸出結果
print("Reduced Model (FK-RFE features) Performance:")
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

FK-RFE 選出的特徵: ['LotArea', 'OverallQual', 'YearBuilt', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea']
Reduced Model (FK-RFE features) Performance:
MSE: 831159261.9555
MAE: 19090.2232
RMSE: 28829.8328
R² Score: 0.8916


In [444]:
print(X_train.select_dtypes(include=['object']).columns)

Index([], dtype='object')
